# F2F MNEFun Processing - Main Batch Script
Starting 2020-10-02, forward and inverse models will be saved in different folders, with the surface-based model indicated with a "\_surf" suffix and volume-based with "\_vol".

Starting 2020-01-29, all 7-second processing will be accomplished by calling the 1-second yml file (f2f_params.yml) and making adjustments as needed. Then, for 1-second processing, the yml file should be reloaded. (The previous f2f_params_7sec.yml file will be archived.)

Also at this time, the eSSS step is being adopted, as with the MMF subjects.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Set-up-processing-parameters" data-toc-modified-id="Set-up-processing-parameters-1">Set up processing parameters</a></span></li><li><span><a href="#Make-one-time-adjustments-of-raw-files" data-toc-modified-id="Make-one-time-adjustments-of-raw-files-2">Make one-time adjustments of raw files</a></span></li><li><span><a href="#Detect-Events" data-toc-modified-id="Detect-Events-3">Detect Events</a></span></li><li><span><a href="#Run-SSS" data-toc-modified-id="Run-SSS-4">Run SSS</a></span></li><li><span><a href="#Run-SSP-and-create-first-set-of-epochs" data-toc-modified-id="Run-SSP-and-create-first-set-of-epochs-5">Run SSP and create first set of epochs</a></span></li><li><span><a href="#Create-sensor-level-reports" data-toc-modified-id="Create-sensor-level-reports-6">Create sensor-level reports</a></span></li><li><span><a href="#Create-shorter-&quot;sub-epochs&quot;-from-the-original-events" data-toc-modified-id="Create-shorter-&quot;sub-epochs&quot;-from-the-original-events-7">Create shorter "sub-epochs" from the original events</a></span></li><li><span><a href="#Perform-a-one-time-creation-of-scaled-mri-files" data-toc-modified-id="Perform-a-one-time-creation-of-scaled-mri-files-8">Perform a one-time creation of scaled mri files</a></span></li><li><span><a href="#Create-covariance,-forward-model,-and-inverse-model" data-toc-modified-id="Create-covariance,-forward-model,-and-inverse-model-9">Create covariance, forward model, and inverse model</a></span></li></ul></div>

## Set up processing parameters

In [1]:
%%capture

import os
from functools import partial

import mnefun
import mne.utils

from f2f_score import score, score_split, delete_sssfiles, report_postfunction, fix_mri

In [2]:
# Read in preset mnefun parameters #
# The sws processing parameters are set in ~/.mnefun/mnefun.json.
# params = mnefun.read_params('f2f_params_7sec.yml')  # 2021-01-29: no longer used
params = mnefun.read_params('f2f_params.yml')         # 2021-01-29: now for surf. model
params.score = score

In [3]:
# Set internal processing parameters #
n_cores = os.cpu_count()

params.n_jobs = params.n_jobs_mkl = params.n_jobs_fir = params.n_jobs_resample = min(n_cores, 6)
if mne.utils.get_config('MNE_USE_CUDA') == 'true':   # false for Mac OS, true for Linux
    params.n_jobs_fir = params.n_jobs_resample = 'cuda'

In [4]:
# Set subject parameters #
# 2021-01-29 skipped so far: f2f_014, f2f_022; f2f_026 has been added back
batch_subjects = [['f2f_009', 'f2f_010', 'f2f_011', 'f2f_012', 'f2f_013'],
                  ['f2f_014', 'f2f_015', 'f2f_018', 'f2f_021', 'f2f_022'],
                  ['f2f_024', 'f2f_025', 'f2f_026', 'f2f_027', 'f2f_028'],
                  ['f2f_029', 'f2f_030', 'f2f_034', 'f2f_036', 'f2f_037'],
                  ['f2f_038', 'f2f_102', 'f2f_103', 'f2f_104', 'f2f_106'],
                  ['f2f_107', 'f2f_108', 'f2f_111', 'f2f_112', 'f2f_114'],
                  ['f2f_115', 'f2f_116', 'f2f_117', 'f2f_118', 'f2f_119']]
extra_subjects = []
bad_subjects = ['f2f_041']   # _041 has bad coil fits
mac_subjects = ['f2f_107']
all_subjects = [subj for row in batch_subjects for subj in row]

use_subjects = ['f2f_011']

params.work_dir = os.path.expanduser('~/Data/MEG/f2f')  # on Thicke
# params.work_dir = os.path.expanduser('/Users/neuromac/Data/MEG/MMF_sample/')  # on Mac
params.subjects_dir = os.path.expanduser('~/Data/MEG/anatomy')
params.subjects = use_subjects
params.structurals = use_subjects

In [5]:
# Set source model parameters #
SOURCE_TYPE = 'surf';    # options are 'surf' or 'vol'

if SOURCE_TYPE == 'surf':
    params.src = 'oct6'
    params.forward_dir = 'forward_surf'
    params.inverse_dir = 'inverse_surf'
elif SOURCE_TYPE == 'vol':
    params.src = 'vol5'
    params.forward_dir = 'forward_vol'
    params.inverse_dir = 'inverse_vol'
else:
    raise ValueError('Source type must be "surf" or "vol".')

In [6]:
# Choose which subjects to run, and any test changes #
params.subject_indices = None  #[1]  #None

## Make one-time adjustments of raw files
Changes to meta-data only, to avoid errors in subsequent processing

## Detect Events ##
This is for the longer 7-second epochs.

In [16]:
# For this and the next two sections, make adjustments to the parameters #
params.list_dir = 'lists'   # for 7-second epochs
params.tmax, params.bmax = 7.0, 7.0
params.in_numbers = [13, 15]
params.epochs_prefix = 'All'
params.reject_epochs_by_annot = False

## Run SSS

In [7]:
# Remove files from previous SSS processing #
# This step is required if chpi processing files are causing issues.
clear_chpi, clear_annot = True, True
if any((clear_chpi, clear_annot)):
    delete_sssfiles(params, clear_chpi, clear_annot)

0 cHPI and head position files deleted for 0 of 1 subjects.
0 annotation and max-bad files deleted for 0 of 1 subjects.


In [9]:
mnefun.do_processing(params, do_sss=True, print_status=True)

Running SSS using python. 
  Applying SSS locally using mne-python
    Maxwell filtering subject 1/1 (f2f_011).
      Computing eSSS basis from ERM with 3 components
Opening raw data file /home/sbierer/Data/MEG/f2f/f2f_011/raw_fif/f2f_011_erm_raw.fif...
    Read a total of 11 projection items:
        mag.fif : PCA-v1 (1 x 306)  idle
        mag.fif : PCA-v2 (1 x 306)  idle
        mag.fif : PCA-v3 (1 x 306)  idle
        mag.fif : PCA-v4 (1 x 306)  idle
        mag.fif : PCA-v5 (1 x 306)  idle
        mag.fif : PCA-v6 (1 x 306)  idle
        mag.fif : PCA-v7 (1 x 306)  idle
        grad.fif : PCA-v1 (1 x 306)  idle
        grad.fif : PCA-v2 (1 x 306)  idle
        grad.fif : PCA-v3 (1 x 306)  idle
        grad.fif : PCA-v4 (1 x 306)  idle
    Range : 135000 ... 1090499 =     43.422 ...   350.756 secs
Ready.
        Marking 5 bad MEG channels using mf_prebad['f2f_011']
101 of 101 magnetometer types replaced with T3.


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.5s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.3s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.3s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.3s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.3s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.3s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:   

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:   

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:   

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:   

...

        Using 63/66 harmonic components for  113.300  (45/48 in, 18/18 out)
        Using 64/66 harmonic components for  114.400  (46/48 in, 18/18 out)
        Using 63/66 harmonic components for  115.500  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  116.000  (45/48 in, 18/18 out)
        Projecting  1 intersecting tSSS component  for  116.000 -  120.000 sec (#30/76) (across  8 positions)
        Using 63/66 harmonic components for  116.600  (45/48 in, 18/18 out)
        Using 64/66 harmonic components for  116.999  (46/48 in, 18/18 out)
        Using 63/66 harmonic components for  117.199  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  117.300  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  117.500  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  118.600  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  119.199  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  120.000  

        Using 64/66 harmonic components for  155.100  (46/48 in, 18/18 out)
        Using 64/66 harmonic components for  155.199  (46/48 in, 18/18 out)
        Using 65/66 harmonic components for  155.399  (47/48 in, 18/18 out)
        Using 64/66 harmonic components for  155.500  (46/48 in, 18/18 out)
        Using 65/66 harmonic components for  155.899  (47/48 in, 18/18 out)
        Using 62/66 harmonic components for  156.000  (44/48 in, 18/18 out)
        Projecting  0 intersecting tSSS components for  156.000 -  160.000 sec (#40/76) (across 24 positions)
        Using 65/66 harmonic components for  156.000  (47/48 in, 18/18 out)
        Using 64/66 harmonic components for  156.099  (46/48 in, 18/18 out)
        Using 64/66 harmonic components for  156.299  (46/48 in, 18/18 out)
        Using 64/66 harmonic components for  156.799  (46/48 in, 18/18 out)
        Using 64/66 harmonic components for  156.900  (46/48 in, 18/18 out)
        Using 63/66 harmonic components for  156.999  

        Using 61/66 harmonic components for  191.599  (43/48 in, 18/18 out)
        Using 61/66 harmonic components for  191.700  (43/48 in, 18/18 out)
        Using 61/66 harmonic components for  191.800  (43/48 in, 18/18 out)
        Using 63/66 harmonic components for  192.000  (45/48 in, 18/18 out)
        Projecting  1 intersecting tSSS component  for  192.000 -  196.000 sec (#49/76) (across 12 positions)
        Using 62/66 harmonic components for  192.000  (44/48 in, 18/18 out)
        Using 62/66 harmonic components for  192.799  (44/48 in, 18/18 out)
        Using 62/66 harmonic components for  192.999  (44/48 in, 18/18 out)
        Using 63/66 harmonic components for  193.199  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  193.399  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  193.500  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  193.599  (45/48 in, 18/18 out)
        Using 64/66 harmonic components for  193.800  

        Using 64/66 harmonic components for  233.500  (46/48 in, 18/18 out)
        Using 64/66 harmonic components for  234.600  (46/48 in, 18/18 out)
        Using 64/66 harmonic components for  235.700  (46/48 in, 18/18 out)
        Using 63/66 harmonic components for  236.000  (45/48 in, 18/18 out)
        Projecting  0 intersecting tSSS components for  236.000 -  240.000 sec (#60/76) (across 15 positions)
        Using 64/66 harmonic components for  236.499  (46/48 in, 18/18 out)
        Using 64/66 harmonic components for  236.700  (46/48 in, 18/18 out)
        Using 63/66 harmonic components for  236.799  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  236.900  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  236.999  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  237.199  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  237.399  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  237.599  

        Using 63/66 harmonic components for  284.000  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  285.100  (45/48 in, 18/18 out)
        Using 64/66 harmonic components for  286.000  (46/48 in, 18/18 out)
        Using 64/66 harmonic components for  286.099  (46/48 in, 18/18 out)
        Using 64/66 harmonic components for  286.200  (46/48 in, 18/18 out)
        Using 64/66 harmonic components for  286.700  (46/48 in, 18/18 out)
        Using 64/66 harmonic components for  286.999  (46/48 in, 18/18 out)
        Using 63/66 harmonic components for  287.199  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  287.399  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  287.500  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  287.599  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  287.700  (45/48 in, 18/18 out)
        Using 63/66 harmonic components for  287.800  (45/48 in, 18/18 out)
        Usin

## Run SSP and create first set of epochs
Make sure the parameters in "f2f_params.yml" have been reset above (in __Detect Events__) for the 7-sec settings.

In [10]:
print('Current "params.proj_nums" setting:', params.proj_nums['default'], '\n')

mnefun.do_processing(params, gen_ssp=True, apply_ssp=True, print_status=False)

Current "params.proj_nums" setting: [[2, 2, 0], [0, 0, 0], [3, 3, 0]] 

Preprocessing files. 
  Preprocessing subject 1/1 (f2f_011).
    Clearing bad channels (no file f2f_011/bads/bad_ch_f2f_011_post-sss.txt)
    Loading and filtering 1 file.
Opening raw data file /home/sbierer/Data/MEG/f2f/f2f_011/sss_fif/f2f_011_raw_sss.fif...
    Range : 145500 ... 1720499 =     46.800 ...   553.393 secs
Ready.
Opening raw data file /home/sbierer/Data/MEG/f2f/f2f_011/sss_fif/f2f_011_raw_sss-1.fif...
    Range : 1720500 ... 3182999 =    553.393 ...  1023.802 secs
Ready.
Reading 0 ... 3037499  =      0.000 ...   977.002 secs...


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  12 tasks      | elapsed:    1.1s
[Parallel(n_jobs=6)]: Done  66 tasks      | elapsed:    3.3s
[Parallel(n_jobs=6)]: Done 156 tasks      | elapsed:    6.9s
[Parallel(n_jobs=6)]: Done 282 tasks      | elapsed:   12.0s
[Parallel(n_jobs=6)]: Done 306 out of 306 | elapsed:   12.9s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  18 tasks      | elapsed:    1.0s
[Parallel(n_jobs=6)]: Done 126 tasks      | elapsed:    6.0s
[Parallel(n_jobs=6)]: Done 306 out of 306 | elapsed:   14.1s finished


0 projection items deactivated
No projector specified for this dataset. Please consider the method self.add_proj.
    Computing ECG projectors...Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 5 - 35 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 5.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 4.75 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 35.25 Hz)
- Filter length: 2480 samples (12.400 sec)



[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  18 tasks      | elapsed:    0.1s


0 projection items deactivated
No projector specified for this dataset. Please consider the method self.add_proj.
Using channel ECG061 to identify heart beats.


...

SSP projectors applied...
Not setting metadata
Not setting metadata
977 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 2)
2 projection items activated
Loading data for 977 events and 201 original time points ...
    Rejecting  epoch based on GRAD : ['MEG0123', 'MEG1722', 'MEG1732']
    Rejecting  epoch based on GRAD : ['MEG0242', 'MEG1522', 'MEG1532', 'MEG1542', 'MEG1712']
    Rejecting  epoch based on GRAD : ['MEG1522', 'MEG1542', 'MEG1722']
    Rejecting  epoch based on MAG : ['MEG1541']
    Rejecting  epoch based on GRAD : ['MEG0113', 'MEG0122', 'MEG0132', 'MEG0143']
    Rejecting  epoch based on GRAD : ['MEG0113', 'MEG0112', 'MEG0122', 'MEG0123', 'MEG0132', 'MEG0133', 'MEG0142', 'MEG0313', 'MEG0343']
    Rejecting  epoch based on GRAD : ['MEG0113', 'MEG0123']
    Rejecting  epoch based on GRAD : ['MEG0242', 'MEG0342']
    Rejecting  epoch based on GRAD : ['MEG0112']
    Rejecting  epoch based on GRAD : ['MEG0113', 'MEG0

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  18 tasks      | elapsed:    0.4s
[Parallel(n_jobs=6)]: Done 234 tasks      | elapsed:    4.1s
[Parallel(n_jobs=6)]: Done 305 out of 305 | elapsed:    5.3s finished


2 projection items deactivated
Overwriting existing file.
Writing /home/sbierer/Data/MEG/f2f/f2f_011/sss_pca_fif/f2f_011_erm_allclean_fil80_raw_sss.fif
Closing /home/sbierer/Data/MEG/f2f/f2f_011/sss_pca_fif/f2f_011_erm_allclean_fil80_raw_sss.fif
[done]
    Processing file 1/1.
Opening raw data file /home/sbierer/Data/MEG/f2f/f2f_011/sss_fif/f2f_011_raw_sss.fif...
    Range : 145500 ... 1720499 =     46.800 ...   553.393 secs
Ready.
Opening raw data file /home/sbierer/Data/MEG/f2f/f2f_011/sss_fif/f2f_011_raw_sss-1.fif...
    Range : 1720500 ... 3182999 =    553.393 ...  1023.802 secs
Ready.
Reading 0 ... 3037499  =      0.000 ...   977.002 secs...
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 80 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hann window with 0.0546 passband ripple and 44 dB stopband attenuation
- Upper passband edge: 80.00 Hz
- Upper t

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  12 tasks      | elapsed:    0.7s
[Parallel(n_jobs=6)]: Done  66 tasks      | elapsed:    3.3s
[Parallel(n_jobs=6)]: Done 156 tasks      | elapsed:    8.5s
[Parallel(n_jobs=6)]: Done 282 tasks      | elapsed:   15.9s
[Parallel(n_jobs=6)]: Done 305 out of 305 | elapsed:   17.2s finished


2 projection items deactivated
Overwriting existing file.
Writing /home/sbierer/Data/MEG/f2f/f2f_011/sss_pca_fif/f2f_011_allclean_fil80_raw_sss.fif
Writing /home/sbierer/Data/MEG/f2f/f2f_011/sss_pca_fif/f2f_011_allclean_fil80_raw_sss-1.fif
Closing /home/sbierer/Data/MEG/f2f/f2f_011/sss_pca_fif/f2f_011_allclean_fil80_raw_sss-1.fif
Closing /home/sbierer/Data/MEG/f2f/f2f_011/sss_pca_fif/f2f_011_allclean_fil80_raw_sss.fif
[done]
  (0:01:08.100)
Done


In [17]:
mnefun.do_processing(params, write_epochs=True, print_status=True)

Doing epoch EQ/DQ. 
  Loading raw files for subject f2f_011.
Opening raw data file /home/sbierer/Data/MEG/f2f/f2f_011/sss_pca_fif/f2f_011_allclean_fil80_raw_sss.fif...
    Read a total of 2 projection items:
        meg-ECG--0.08-0.08-PCA-01 (1 x 305)  idle
        meg-ECG--0.08-0.08-PCA-02 (1 x 305)  idle
    Range : 145500 ... 1720499 =     46.800 ...   553.393 secs
Ready.
Opening raw data file /home/sbierer/Data/MEG/f2f/f2f_011/sss_pca_fif/f2f_011_allclean_fil80_raw_sss-1.fif...
    Read a total of 2 projection items:
        meg-ECG--0.08-0.08-PCA-01 (1 x 305)  idle
        meg-ECG--0.08-0.08-PCA-02 (1 x 305)  idle
    Range : 1720500 ... 3182999 =    553.393 ...  1023.802 secs
Ready.
    Epoching data (decim=10 -> sfreq=310.9 Hz).
    Computing autoreject thresholdsNot setting metadata
Not setting metadata
180 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 2)
2 projection items activated
Loading data for 180 events and

In [12]:
raise SystemExit("Stopping after SSP.")

SystemExit: Stopping after SSP.

/home/sbierer/miniconda3/envs/mne_dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Create sensor-level reports
OK to run with above params (7-seconds) or subsequent params (x-seconds).  However, these reports only reflect epoch duration for the "Sensor" section time snippets and for the "Drop Log".  
__Saved directories__  
qa_2020Sept: original 7-second epochs  
qa_1sec_2020Sept: (4 subjects) 1-second epochs
qa_esss_2021Jan: (partial) 1-second epochs with eSSS  
qa_esss_2021Feb: (partial) 1-second epochs with eSSS; concerned about epoch amp-based rejections since last run  

In [12]:
# Set up the report structure #
# params.report = {}   # start from scratch then set unwanted ones as False
# params.report.update(source_alignment=False, covariance=False,
#                      bem=False, snr=False, source=False)

t_points = [np.round(params.tmax*x, 2) for x in (1/3, 2/3)]

params.report.update(
    sensor=[{'analysis':'Each', 'name':'talk', 'times':t_points, 'proj':True},
            {'analysis':'Each', 'name':'ignore', 'times':t_points, 'proj':True}]
    )

In [13]:
# Optional: Extra parameters for the post-function feature #
save_dir = '/home/sbierer/Projects/ILABS/Analyses/F2F/results/qa_esss_2021Feb'
postfun = partial(report_postfunction, save_path = save_dir,
                  save_prefix = '', save_postfix = '')

params.report.update(post_fun = postfun)

In [14]:
print('Current "params.list_dir" setting:', params.list_dir, '\n')

mnefun.do_processing(params, gen_report=True, print_status=False)

Current "params.list_dir" setting: lists_1sec 

Generating HTML Reports. 


In /home/sbierer/miniconda3/envs/mne_dev/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/sbierer/miniconda3/envs/mne_dev/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/sbierer/miniconda3/envs/mne_dev/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/sbierer/miniconda3/envs/mne_dev/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.


  Processing subject 1/1 (f2f_011)
Opening raw data file /home/sbierer/Data/MEG/f2f/f2f_011/raw_fif/f2f_011_raw.fif...
    Read a total of 11 projection items:
        mag.fif : PCA-v1 (1 x 306)  idle
        mag.fif : PCA-v2 (1 x 306)  idle
        mag.fif : PCA-v3 (1 x 306)  idle
        mag.fif : PCA-v4 (1 x 306)  idle
        mag.fif : PCA-v5 (1 x 306)  idle
        mag.fif : PCA-v6 (1 x 306)  idle
        mag.fif : PCA-v7 (1 x 306)  idle
        grad.fif : PCA-v1 (1 x 306)  idle
        grad.fif : PCA-v2 (1 x 306)  idle
        grad.fif : PCA-v3 (1 x 306)  idle
        grad.fif : PCA-v4 (1 x 306)  idle
    Range : 145500 ... 1691999 =     46.800 ...   544.226 secs
Ready.
Opening raw data file /home/sbierer/Data/MEG/f2f/f2f_011/raw_fif/f2f_011_raw-1.fif...
    Read a total of 11 projection items:
        mag.fif : PCA-v1 (1 x 306)  idle
        mag.fif : PCA-v2 (1 x 306)  idle
        mag.fif : PCA-v3 (1 x 306)  idle
        mag.fif : PCA-v4 (1 x 306)  idle
        mag.fif : PCA-v5

Effective window size : 2.635 (s)
Effective window size : 2.635 (s)
Effective window size : 2.635 (s)
Setting 106821 of 955500 (11.18%) samples to NaN, retaining 848679 (88.82%) samples.
Effective window size : 2.635 (s)
Setting 106821 of 955500 (11.18%) samples to NaN, retaining 848679 (88.82%) samples.
Effective window size : 2.635 (s)
102.8 sec
    SSP topomaps ...         Read a total of 3 projection items:
        meg-Raw-0.000-307.335-PCA-01 (1 x 300)  idle
        meg-Raw-0.000-307.335-PCA-02 (1 x 300)  idle
        meg-Raw-0.000-307.335-PCA-03 (1 x 300)  idle
    Read a total of 2 projection items:
        meg-ECG--0.08-0.08-PCA-01 (1 x 305)  idle
        meg-ECG--0.08-0.08-PCA-02 (1 x 305)  idle
Reading /home/sbierer/Data/MEG/f2f/f2f_011/sss_pca_fif/preproc_ecg-epo.fif ...
    Found the data of interest:
        t =     -80.00 ...      80.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
1270 matching events found
No baseline correct

  (0:03:37.002)
Done


## Create shorter "sub-epochs" from the original events
Originally, 1-second events and epochs were created.  The code was updated and retested on the Mac in Jan 2020.

In [18]:
# Reload and update the parameters #
# Using the new 'score_split()' function to obtain short interval "sub-events"
subject_indices = params.subject_indices

params = mnefun.read_params('f2f_params.yml')
params.score = partial(score_split, old_dir='lists', id_dict={13:23, 15:25},
                       tlimits=(0,7.5), dur=1.0, step=0.5)

n_cores = os.cpu_count()
params.n_jobs = params.n_jobs_mkl = params.n_jobs_fir = params.n_jobs_resample = min(n_cores, 6)
if mne.utils.get_config('MNE_USE_CUDA') == 'true':   # false for Mac OS, true for Linux
    params.n_jobs_fir = params.n_jobs_resample = 'cuda'
    
params.work_dir = os.path.expanduser('~/Data/MEG/f2f')  # on Thicke
# params.work_dir = os.path.expanduser('~/Data/MEG/MMF_sample')  # on Mac
params.subjects_dir = os.path.expanduser('~/Data/MEG/anatomy')
params.subjects = use_subjects
params.structurals = use_subjects
params.subject_indices = subject_indices

# Set source model parameters #
SOURCE_TYPE = 'surf';    # options are 'surf' or 'vol'

if SOURCE_TYPE == 'surf':
    params.src = 'oct6'
    params.forward_dir = 'forward_surf'
    params.inverse_dir = 'inverse_surf'
elif SOURCE_TYPE == 'vol':
    params.src = 'vol5'
    params.forward_dir = 'forward_vol'
    params.inverse_dir = 'inverse_vol'
else:
    raise ValueError('Source type must be "surf" or "vol".')

In [19]:
# Generate 1-sec epochs, with files named using the .epochs_prefix parameter.
mnefun.do_processing(params, write_epochs=True, print_status=False)

Doing epoch EQ/DQ. 
  Loading raw files for subject f2f_011.
Opening raw data file /home/sbierer/Data/MEG/f2f/f2f_011/sss_pca_fif/f2f_011_allclean_fil80_raw_sss.fif...
    Read a total of 2 projection items:
        meg-ECG--0.08-0.08-PCA-01 (1 x 305)  idle
        meg-ECG--0.08-0.08-PCA-02 (1 x 305)  idle
    Range : 145500 ... 1720499 =     46.800 ...   553.393 secs
Ready.
Opening raw data file /home/sbierer/Data/MEG/f2f/f2f_011/sss_pca_fif/f2f_011_allclean_fil80_raw_sss-1.fif...
    Read a total of 2 projection items:
        meg-ECG--0.08-0.08-PCA-01 (1 x 305)  idle
        meg-ECG--0.08-0.08-PCA-02 (1 x 305)  idle
    Range : 1720500 ... 3182999 =    553.393 ...  1023.802 secs
Ready.
    Epoching data (decim=10 -> sfreq=310.9 Hz).
    Computing autoreject thresholdsNot setting metadata
Not setting metadata
1246 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 2)
2 projection items activated
Loading data for 1246 events a

    Rejecting  epoch based on GRAD : ['MEG0122', 'MEG0132', 'MEG0133', 'MEG0142', 'MEG0213', 'MEG0212', 'MEG0222', 'MEG0223', 'MEG0232', 'MEG0233', 'MEG0242', 'MEG0342', 'MEG1512', 'MEG1513', 'MEG1542', 'MEG1612']
    Rejecting  epoch based on GRAD : ['MEG0113', 'MEG0122', 'MEG0132', 'MEG0133', 'MEG0143', 'MEG0142', 'MEG0213', 'MEG0212', 'MEG0222', 'MEG0223', 'MEG0232', 'MEG0233', 'MEG0242', 'MEG0342', 'MEG1512', 'MEG1513', 'MEG1542', 'MEG1612']
    Rejecting  epoch based on GRAD : ['MEG0113', 'MEG0132', 'MEG0143', 'MEG0213', 'MEG0212', 'MEG0222', 'MEG0232', 'MEG0242', 'MEG0342', 'MEG1513']
    Rejecting  epoch based on GRAD : ['MEG0132', 'MEG0212', 'MEG0222', 'MEG0232', 'MEG0242', 'MEG0342', 'MEG1513']
    Rejecting  epoch based on GRAD : ['MEG0132', 'MEG0222', 'MEG0242', 'MEG0342']
    Rejecting  epoch based on GRAD : ['MEG0113', 'MEG0132']
    Rejecting  epoch based on GRAD : ['MEG0242', 'MEG0342']
    Rejecting  epoch based on GRAD : ['MEG0242', 'MEG0342']
    Rejecting  epoch base

    Rejecting  epoch based on GRAD : ['MEG1522']
    Rejecting  epoch based on GRAD : ['MEG0113', 'MEG0122', 'MEG0132', 'MEG0143', 'MEG0213', 'MEG0212', 'MEG0243', 'MEG0343', 'MEG1512', 'MEG1513', 'MEG1522', 'MEG1613']
    Rejecting  epoch based on GRAD : ['MEG0113', 'MEG0122', 'MEG0132', 'MEG0143', 'MEG0213', 'MEG0243', 'MEG0343', 'MEG1512', 'MEG1522', 'MEG1613']
    Rejecting  epoch based on GRAD : ['MEG0113', 'MEG0122', 'MEG0132', 'MEG0143', 'MEG0213', 'MEG0243', 'MEG0343', 'MEG1512', 'MEG1513', 'MEG1522']
    Rejecting  epoch based on GRAD : ['MEG0113', 'MEG0122', 'MEG0132', 'MEG0143', 'MEG0213', 'MEG0243', 'MEG1512', 'MEG1513', 'MEG1522']
    Rejecting  epoch based on GRAD : ['MEG0113', 'MEG0132', 'MEG0143', 'MEG0213', 'MEG0243', 'MEG0343', 'MEG1512', 'MEG1522', 'MEG1613']
    Rejecting  epoch based on GRAD : ['MEG0113', 'MEG0132', 'MEG0143', 'MEG0213', 'MEG0212', 'MEG0243', 'MEG0242', 'MEG1512', 'MEG1522', 'MEG1613']
    Rejecting  epoch based on GRAD : ['MEG0113', 'MEG0122', 'ME

## Perform a one-time creation of scaled mri files
Some of the anatomy folders I received from MC did not have an "mri" folder, required for volumetric models, so this was the suggested fix.  
Also, in Explore_STCMethods, I made a one-time creation of the volumetric source space for the 14-month surrogate brain, at _bem/14mo_surr-vol-5-src.fif_.

## Create covariance, forward model, and inverse model
2020-09-25: Volumetric model with 5 mm spacing. (The 'src_pos' parameter may not do anything, but I'm changing it to 5.0 after this run.)  
2020-10-02: Volumetric and surface models are saved in different folders from now on.

In [13]:
mnefun.do_processing(params, gen_covs=True, print_status=False)

Generating covariances. 
  Subject  1/ 3...Reading /home/sbierer/Data/MEG/f2f/f2f_014/epochs/All_1sec_80-sss_f2f_014-epo.fif ...
    Read a total of 2 projection items:
        meg-ECG--0.08-0.08-PCA-01 (1 x 305)  idle
        meg-ECG--0.08-0.08-PCA-02 (1 x 305)  idle
    Found the data of interest:
        t =       0.00 ...     997.11 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
385 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 2)
2 projection items activated
Projections have already been applied. Setting proj attribute to True.
 : MEG rank 45Opening raw data file /home/sbierer/Data/MEG/f2f/f2f_014/sss_pca_fif/f2f_014_erm_allclean_fil80_raw_sss.fif...
    Read a total of 2 projection items:
        meg-ECG--0.08-0.08-PCA-01 (1 x 305)  idle
        meg-ECG--0.08-0.08-PCA-02 (1 x 305)  idle
    Range : 36000 ... 370999 =     36.000 ...   370.999 secs
Ready.
Reading 0 ... 334999  =      0.

In [14]:
mnefun.do_processing(params, gen_fwd=True, print_status=False)

Generating forward models. 
    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
  Creating forward solution(s) using a 1-layer BEM for f2f_014...Using surface from /media/sbierer/smbdata/MEG/anatomy/f2f_014/bem/f2f_014-head.fif.
 (dig<->MRI 4.4 mm)
Opening raw data file /home/sbierer/Data/MEG/f2f/f2f_014/sss_pca_fif/f2f_014_allclean_fil80_raw_sss.fif...
    Read a total of 2 projection items:
        meg-ECG--0.08-0.08-PCA-01 (1 x 305)  idle
        meg-ECG--0.08-0.08-PCA-02 (1 x 305)  idle
    Range : 217500 ... 1792499 =     69.958 ...   576.552 secs
Ready.
Opening raw data file /media/sbierer/smbdata/MEG/f2f/f2f_014/sss_pca_fif/f2f_014_allclean_fil80_raw_sss-1.fif...
    Read a total of 2 projection items:
        meg-ECG--0.08-0.08-PCA-01 (1 x 305)  idle
        meg-ECG--0.08-0.08-PCA-02 (1 x 305)  idle
    Range : 1792500 ... 2575499 =    576.552 ...   828.401 secs
Ready.
Source space          : <SourceSpaces: [<surface (lh), 

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


    Skipping interior check for 1940 sources that fit inside a sphere of radius   43.0 mm
    Skipping solid angle check for 0 points using Qhull


[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:    1.6s remaining:    1.6s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    1.6s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.



Setting up compensation data...
    No compensation set. Nothing more to do.

Composing the field computation matrix...


...

Computing MEG at 8196 source locations (free orientations)...


[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    0.2s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:    1.2s remaining:    1.2s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    1.6s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.



Finished.
Overwriting existing file.
    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written


[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:    0.4s remaining:    0.4s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    0.4s finished


  (0:00:38.696)
Done


In [15]:
mnefun.do_processing(params, gen_inv=True, print_status=True)

Generating inverse solutions. 
  Subject f2f_014Reading /home/sbierer/Data/MEG/f2f/f2f_014/epochs/All_1sec_80-sss_f2f_014-epo.fif ...
    Read a total of 2 projection items:
        meg-ECG--0.08-0.08-PCA-01 (1 x 305)  idle
        meg-ECG--0.08-0.08-PCA-02 (1 x 305)  idle
    Found the data of interest:
        t =       0.00 ...     997.11 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
385 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 2)
2 projection items activated
    305 x 305 full covariance (kind = 1) found.
    Read a total of 2 projection items:
        meg-ECG--0.08-0.08-PCA-01 (1 x 305)  idle
        meg-ECG--0.08-0.08-PCA-02 (1 x 305)  idle
Opening raw data file /home/sbierer/Data/MEG/f2f/f2f_014/sss_pca_fif/f2f_014_allclean_fil80_raw_sss.fif...
    Read a total of 2 projection items:
        meg-ECG--0.08-0.08-PCA-01 (1 x 305)  idle
        meg-ECG--0.08-0.08-PCA-02 (1 x 305)  

/home/sbierer/CodeBase/mnefun/mnefun/_inverse.py:123: RuntimeWarning: Something went wrong in the data-driven estimation of the data rank as it exceeds the theoretical rank from the info (45 > 43). Consider setting rank to "auto" or setting it explicitly as an integer.
  **kwargs)


Write inverse operator decomposition in /home/sbierer/Data/MEG/f2f/f2f_014/inverse_surf/f2f_014-80-sss-meg-free-inv.fif...
    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written
    Writing inverse operator info...
    Writing noise covariance matrix.
    Writing source covariance matrix.
    Writing orientation priors.
    [done]
Write inverse operator decomposition in /home/sbierer/Data/MEG/f2f/f2f_014/inverse_surf/f2f_014-80-sss-meg-fixed-inv.fif...
    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written
    Writing inverse operator info...
    Writing noise covariance matrix.
    Writing source covariance matrix.
    Writing orientation priors.
    [done]


/home/sbierer/CodeBase/mnefun/mnefun/_inverse.py:123: RuntimeWarning: Something went wrong in the data-driven estimation of the data rank as it exceeds the theoretical rank from the info (45 > 43). Consider setting rank to "auto" or setting it explicitly as an integer.
  **kwargs)
/home/sbierer/CodeBase/mnefun/mnefun/_inverse.py:123: RuntimeWarning: Something went wrong in the data-driven estimation of the data rank as it exceeds the theoretical rank from the info (45 > 43). Consider setting rank to "auto" or setting it explicitly as an integer.
  **kwargs)


Write inverse operator decomposition in /home/sbierer/Data/MEG/f2f/f2f_014/inverse_surf/f2f_014-80-sss-meg-inv.fif...
    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written
    Writing inverse operator info...
    Writing noise covariance matrix.
    Writing source covariance matrix.
    Writing orientation priors.
    [done]
    305 out of 305 channels remain after picking


...

Write inverse operator decomposition in /home/sbierer/Data/MEG/f2f/f2f_026/inverse_surf/f2f_026-80-sss-meg-inv.fif...
    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written
    Writing inverse operator info...
    Writing noise covariance matrix.
    Writing source covariance matrix.
    Writing orientation priors.
    [done]
    306 out of 306 channels remain after picking


/home/sbierer/CodeBase/mnefun/mnefun/_inverse.py:123: RuntimeWarning: Something went wrong in the data-driven estimation of the data rank as it exceeds the theoretical rank from the info (46 > 41). Consider setting rank to "auto" or setting it explicitly as an integer.
  **kwargs)


Write inverse operator decomposition in /home/sbierer/Data/MEG/f2f/f2f_026/inverse_surf/f2f_026-meg-erm-free-inv.fif...
    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written
    Writing inverse operator info...
    Writing noise covariance matrix.
    Writing source covariance matrix.
    Writing orientation priors.
    [done]
Write inverse operator decomposition in /home/sbierer/Data/MEG/f2f/f2f_026/inverse_surf/f2f_026-meg-erm-fixed-inv.fif...
    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written
    Writing inverse operator info...
    Writing noise covariance matrix.
    Writing source covariance matrix.
    Writing orientation priors.
    [done]


/home/sbierer/CodeBase/mnefun/mnefun/_inverse.py:123: RuntimeWarning: Something went wrong in the data-driven estimation of the data rank as it exceeds the theoretical rank from the info (46 > 41). Consider setting rank to "auto" or setting it explicitly as an integer.
  **kwargs)
/home/sbierer/CodeBase/mnefun/mnefun/_inverse.py:123: RuntimeWarning: Something went wrong in the data-driven estimation of the data rank as it exceeds the theoretical rank from the info (46 > 41). Consider setting rank to "auto" or setting it explicitly as an integer.
  **kwargs)


Write inverse operator decomposition in /home/sbierer/Data/MEG/f2f/f2f_026/inverse_surf/f2f_026-meg-erm-inv.fif...
    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written
    Writing inverse operator info...
    Writing noise covariance matrix.
    Writing source covariance matrix.
    Writing orientation priors.
    [done]

  (0:00:14.089)
Status. 
Opening raw data file /home/sbierer/Data/MEG/f2f/f2f_014/sss_fif/f2f_014_raw_sss.fif...
    Range : 217500 ... 1792499 =     69.958 ...   576.552 secs
Ready.
Opening raw data file /media/sbierer/smbdata/MEG/f2f/f2f_014/sss_fif/f2f_014_raw_sss-1.fif...
    Range : 1792500 ... 2575499 =    576.552 ...   828.401 secs
Ready.
Opening raw data file /home/sbierer/Data/MEG/f2f/f2f_014/sss_fif/f2f_014_erm_raw_sss.fif...
    Range : 36000 ... 370999 =     36.000 ...   370.999 secs
Ready.
Opening raw data file /home/sbierer/Data/MEG/f2f/f2f_014/sss_pca_fif/f2f_014_allclean_fil80_raw_sss.fif...
    Read